In [1]:
!pip install transformers torch scikit-learn scipy matplotlib seaborn tqdm

In [2]:
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [18]:
# =============================================================================
# CONFIGURATION
# =============================================================================

STIMULI_PATH = '/kaggle/input/fmri-language-dataset/stimuli.txt'  
FMRI_PATH = '/kaggle/input/fmri-language-dataset/text_fmri_data.pkl'

ROI_NAMES = ['language_left', 'language_right', 'vision_left', 'vision_right', 'task_positive_left', 'task_positive_right', 'dmn_left', 'dmn_right']

N_FOLDS = 5
ALPHA = 1.0
RANDOM_STATE = 42


In [12]:
# =============================================================================
# LOAD DATA
# =============================================================================

print("Loading data...")

try:
    with open(STIMULI_PATH, 'r', encoding='utf-8') as f:
        sentences = [line.strip() for line in f if line.strip()]
    print(f" Loaded {len(sentences)} sentences")
except Exception as e:
    print(f" Error loading stimuli: {e}")
    print("Please update STIMULI_PATH with correct path")
    raise

try:
    with open(FMRI_PATH, 'rb') as f:
        fmri_data = pickle.load(f)
        print(" Loaded fMRI data dictionary.")
        print(f"ROIs available: {list(fmri_data.keys())}")
        print(f"Shape of {ROI_NAMES[0]}: {fmri_data[ROI_NAMES[0]].shape}")
except Exception as e:
    print(f" Error loading fMRI data: {e}")
    print("Please update FMRI_PATH with correct path")
    raise

Loading data...
 Loaded 627 sentences
 Loaded fMRI data dictionary.
ROIs available: ['language_left', 'language_right', 'vision_left', 'vision_right', 'task_positive_left', 'task_positive_right', 'dmn_left', 'dmn_right']
Shape of language_left: (627, 11578)


In [13]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def compute_2v2_accuracy(Y_true, Y_pred):
    """Compute 2v2 accuracy metric"""
    N = len(Y_true)
    correct = 0
    total = 0
    
    for i in range(N):
        for j in range(i + 1, N):
            cos_ii = 1 - cosine(Y_true[i], Y_pred[i])
            cos_jj = 1 - cosine(Y_true[j], Y_pred[j])
            cos_ij = 1 - cosine(Y_true[i], Y_pred[j])
            cos_ji = 1 - cosine(Y_true[j], Y_pred[i])
            
            if (cos_ii + cos_jj) > (cos_ij + cos_ji):
                correct += 1
            total += 1
    
    return correct / total if total > 0 else 0.0


def compute_pearson_correlation(Y_true, Y_pred):
    """Compute average Pearson correlation"""
    correlations = []
    for i in range(len(Y_true)):
        corr, _ = pearsonr(Y_true[i], Y_pred[i])
        if not np.isnan(corr):
            correlations.append(corr)
    return np.mean(correlations) if correlations else 0.0


def cross_validate_model(X, Y, n_folds=5, alpha=1.0):
    """K-fold cross-validation with Ridge regression"""
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    
    fold_2v2 = []
    fold_pearson = []
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        
        model = Ridge(alpha=alpha)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        
        fold_2v2.append(compute_2v2_accuracy(Y_test, Y_pred))
        fold_pearson.append(compute_pearson_correlation(Y_test, Y_pred))
    
    return np.mean(fold_2v2), np.mean(fold_pearson)

In [14]:
# =============================================================================
# EXTRACT EMBEDDINGS
# =============================================================================

print("Extracting sentence embeddings...")

# Try to import transformers
try:
    import torch
    from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
    TRANSFORMERS_AVAILABLE = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f" Using device: {device}")
except ImportError:
    print(" Transformers not installed ")
    TRANSFORMERS_AVAILABLE = False

embeddings_dict = {}

# -----------------------------------------------------------------------------
# BERT
# -----------------------------------------------------------------------------
if TRANSFORMERS_AVAILABLE:
    print("\n  Extracting BERT embeddings...")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased').to(device)
    model.eval()
    
    bert_cls, bert_pooled = [], []
    
    with torch.no_grad():
        for i, sentence in enumerate(sentences):
            if i % 100 == 0:
                print(f"    Progress: {i}/{len(sentences)}", end='\r')
            
            inputs = tokenizer(sentence, return_tensors='pt', 
                              truncation=True, max_length=512).to(device)
            outputs = model(**inputs)
            
            # CLS
            bert_cls.append(outputs.last_hidden_state[0, 0, :].cpu().numpy())
            
            # Pooled
            mask = inputs['attention_mask'][0].cpu().numpy()
            tokens = outputs.last_hidden_state[0].cpu().numpy()
            pooled = np.sum(tokens * mask[:, np.newaxis], axis=0) / np.sum(mask)
            bert_pooled.append(pooled)
    
    embeddings_dict['BERT_CLS'] = np.array(bert_cls)
    embeddings_dict['BERT_Pooled'] = np.array(bert_pooled)
    print(f"    ✓ BERT CLS: {embeddings_dict['BERT_CLS'].shape}")
    print(f"    ✓ BERT Pooled: {embeddings_dict['BERT_Pooled'].shape}")
    
    del model
    torch.cuda.empty_cache()

# -----------------------------------------------------------------------------
# RoBERTa
# -----------------------------------------------------------------------------
if TRANSFORMERS_AVAILABLE:
    print("\n  Extracting RoBERTa embeddings...")
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaModel.from_pretrained('roberta-base').to(device)
    model.eval()
    
    roberta_cls, roberta_pooled = [], []
    
    with torch.no_grad():
        for i, sentence in enumerate(sentences):
            if i % 100 == 0:
                print(f"    Progress: {i}/{len(sentences)}", end='\r')
            
            inputs = tokenizer(sentence, return_tensors='pt', 
                              truncation=True, max_length=512).to(device)
            outputs = model(**inputs)
            
            # CLS
            roberta_cls.append(outputs.last_hidden_state[0, 0, :].cpu().numpy())
            
            # Pooled
            mask = inputs['attention_mask'][0].cpu().numpy()
            tokens = outputs.last_hidden_state[0].cpu().numpy()
            pooled = np.sum(tokens * mask[:, np.newaxis], axis=0) / np.sum(mask)
            roberta_pooled.append(pooled)
    
    embeddings_dict['RoBERTa_CLS'] = np.array(roberta_cls)
    embeddings_dict['RoBERTa_Pooled'] = np.array(roberta_pooled)
    print(f"    ✓ RoBERTa CLS: {embeddings_dict['RoBERTa_CLS'].shape}")
    print(f"    ✓ RoBERTa Pooled: {embeddings_dict['RoBERTa_Pooled'].shape}")
    
    del model
    torch.cuda.empty_cache()

# -----------------------------------------------------------------------------
# GloVe
# -----------------------------------------------------------------------------
print("\n  Extracting GloVe embeddings...")

# Download GloVe
import os
import urllib.request
import zipfile

GLOVE_DIR = '/kaggle/working/glove'
GLOVE_FILE = 'glove.6B.300d.txt'
GLOVE_PATH = os.path.join(GLOVE_DIR, GLOVE_FILE)

if not os.path.exists(GLOVE_PATH):
    print("    Downloading GloVe (this may take a few minutes)...")
    os.makedirs(GLOVE_DIR, exist_ok=True)
    
    url = 'http://nlp.stanford.edu/data/glove.6B.zip'
    zip_path = os.path.join(GLOVE_DIR, 'glove.6B.zip')
    
    try:
        urllib.request.urlretrieve(url, zip_path)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(GLOVE_DIR)
        print("    ✓ GloVe downloaded")
    except Exception as e:
        print(f"    ✗ Error downloading GloVe: {e}")
        raise

# Load GloVe
print("    Loading GloVe vectors...")
glove_dict = {}
with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i % 50000 == 0:
            print(f"      Loading: {i} vectors...", end='\r')
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_dict[word] = vector

print(f"\n    ✓ Loaded {len(glove_dict)} GloVe vectors")

# Compute sentence embeddings
glove_embeddings = []
for i, sentence in enumerate(sentences):
    if i % 100 == 0:
        print(f"    Progress: {i}/{len(sentences)}", end='\r')
    
    words = sentence.lower().split()
    vectors = []
    for word in words:
        word = word.strip('.,!?;:()[]{}"\'-')
        if word in glove_dict:
            vectors.append(glove_dict[word])
    
    if vectors:
        glove_embeddings.append(np.mean(vectors, axis=0))
    else:
        glove_embeddings.append(np.zeros(300))

embeddings_dict['GloVe'] = np.array(glove_embeddings)
print(f"\n    ✓ GloVe: {embeddings_dict['GloVe'].shape}")

Extracting sentence embeddings...


2026-02-02 20:16:23.713307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770063384.071059      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770063384.164883      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770063384.981776      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770063384.981820      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770063384.981823      55 computation_placer.cc:177] computation placer alr

 Using device: cuda

  Extracting BERT embeddings...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    ✓ RoBERTa CLS: (627, 768)
    ✓ RoBERTa Pooled: (627, 768)

  Extracting GloVe embeddings...
    ✓ GloVe downloaded
    Loading GloVe vectors...
      Loading: 350000 vectors...
    ✓ Loaded 400000 GloVe vectors
    Progress: 600/627
    ✓ GloVe: (627, 300)


In [20]:
# =============================================================================
# DECODER
# =============================================================================

print("Running DECODER (fMRI -> Embeddings)...")

decoder_results = []

for roi_name, fmri_roi_data in fmri_data.items():
    print(f"\n  ROI: {roi_name}")
    
    # fmri_roi_data is already the array for this region (Shape: N_samples x N_voxels)
    X_fmri = fmri_roi_data 
    
    for emb_name, embeddings in embeddings_dict.items():
        # Ensure dimensions match (sometimes fMRI data might have slightly different lengths)
        min_len = min(len(X_fmri), len(embeddings))
        X_curr = X_fmri[:min_len]
        Y_curr = embeddings[:min_len]
        
        acc, pc = cross_validate_model(X_curr, Y_curr, N_FOLDS, ALPHA)
        
        decoder_results.append({
            'ROI': roi_name,
            'Embedding': emb_name,
            '2v2_Accuracy': acc,
            'Pearson_Correlation': pc
        })
        print(f"    {emb_name:15s} - 2v2: {acc:.4f}, PC: {pc:.4f}")

decoder_df = pd.DataFrame(decoder_results)

Running DECODER (fMRI -> Embeddings)...

  ROI: language_left
    BERT_CLS        - 2v2: 0.9441, PC: 0.8251
    BERT_Pooled     - 2v2: 0.9809, PC: 0.7034
    RoBERTa_CLS     - 2v2: 0.9759, PC: 0.9980
    RoBERTa_Pooled  - 2v2: 0.9686, PC: 0.9759
    GloVe           - 2v2: 0.9462, PC: 0.7705

  ROI: language_right
    BERT_CLS        - 2v2: 0.9255, PC: 0.8237
    BERT_Pooled     - 2v2: 0.9734, PC: 0.7009
    RoBERTa_CLS     - 2v2: 0.9756, PC: 0.9980
    RoBERTa_Pooled  - 2v2: 0.9552, PC: 0.9759
    GloVe           - 2v2: 0.9302, PC: 0.7692

  ROI: vision_left
    BERT_CLS        - 2v2: 0.9245, PC: 0.8179
    BERT_Pooled     - 2v2: 0.9749, PC: 0.6938
    RoBERTa_CLS     - 2v2: 0.9693, PC: 0.9979
    RoBERTa_Pooled  - 2v2: 0.9619, PC: 0.9755
    GloVe           - 2v2: 0.9356, PC: 0.7677

  ROI: vision_right
    BERT_CLS        - 2v2: 0.8879, PC: 0.7978
    BERT_Pooled     - 2v2: 0.9375, PC: 0.6597
    RoBERTa_CLS     - 2v2: 0.9329, PC: 0.9976
    RoBERTa_Pooled  - 2v2: 0.9164, PC: 0.9717


In [21]:
# =============================================================================
# TASK 1.2: ENCODER
# =============================================================================

print(" Running ENCODER (Embeddings -> fMRI)...")

encoder_results = []

for roi_name, fmri_roi_data in fmri_data.items():
    print(f"\n  ROI: {roi_name}")
    
    # fmri_roi_data is the TARGET here
    Y_fmri = fmri_roi_data
    
    for emb_name, embeddings in embeddings_dict.items():
        # Ensure dimensions match
        min_len = min(len(Y_fmri), len(embeddings))
        X_curr = embeddings[:min_len]
        Y_curr = Y_fmri[:min_len]
        
        acc, pc = cross_validate_model(X_curr, Y_curr, N_FOLDS, ALPHA)
        
        encoder_results.append({
            'ROI': roi_name,
            'Embedding': emb_name,
            '2v2_Accuracy': acc,
            'Pearson_Correlation': pc
        })
        print(f"    {emb_name:15s} - 2v2: {acc:.4f}, PC: {pc:.4f}")

encoder_df = pd.DataFrame(encoder_results)

 Running ENCODER (Embeddings -> fMRI)...

  ROI: language_left
    BERT_CLS        - 2v2: 0.6914, PC: 0.4813
    BERT_Pooled     - 2v2: 0.7159, PC: 0.5002
    RoBERTa_CLS     - 2v2: 0.8344, PC: 0.6722
    RoBERTa_Pooled  - 2v2: 0.8097, PC: 0.6180
    GloVe           - 2v2: 0.7675, PC: 0.6112

  ROI: language_right
    BERT_CLS        - 2v2: 0.6814, PC: 0.2948
    BERT_Pooled     - 2v2: 0.7092, PC: 0.3141
    RoBERTa_CLS     - 2v2: 0.7909, PC: 0.4888
    RoBERTa_Pooled  - 2v2: 0.7718, PC: 0.4199
    GloVe           - 2v2: 0.7503, PC: 0.4178

  ROI: vision_left
    BERT_CLS        - 2v2: 0.7317, PC: 0.6490
    BERT_Pooled     - 2v2: 0.7722, PC: 0.6703
    RoBERTa_CLS     - 2v2: 0.8140, PC: 0.7853
    RoBERTa_Pooled  - 2v2: 0.8175, PC: 0.7554
    GloVe           - 2v2: 0.7630, PC: 0.7380

  ROI: vision_right
    BERT_CLS        - 2v2: 0.7259, PC: 0.6404
    BERT_Pooled     - 2v2: 0.7561, PC: 0.6628
    RoBERTa_CLS     - 2v2: 0.7975, PC: 0.7797
    RoBERTa_Pooled  - 2v2: 0.7958, PC: 0.7470

In [23]:
# =============================================================================
# SAVE & VISUALIZE
# =============================================================================

print(" Saving results and creating plots...")

# Save CSVs
decoder_df.to_csv('decoder_results.csv', index=False)
encoder_df.to_csv('encoder_results.csv', index=False)
print("  ✓ Saved decoder_results.csv")
print("  ✓ Saved encoder_results.csv")

# Create plots
sns.set_style("whitegrid")

# Decoder plots
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

decoder_2v2 = decoder_df.pivot(index='ROI', columns='Embedding', values='2v2_Accuracy')
decoder_2v2.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('DECODER: 2v2 Accuracy', fontsize=14, fontweight='bold')
axes[0].set_ylabel('2v2 Accuracy')
axes[0].legend(title='Embedding', bbox_to_anchor=(1.05, 1))
axes[0].tick_params(axis='x', rotation=45)

decoder_pc = decoder_df.pivot(index='ROI', columns='Embedding', values='Pearson_Correlation')
decoder_pc.plot(kind='bar', ax=axes[1], width=0.8)
axes[1].set_title('DECODER: Pearson Correlation', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Pearson Correlation')
axes[1].legend(title='Embedding', bbox_to_anchor=(1.05, 1))
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('decoder_results.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved decoder_results.png")
plt.close()

# Encoder plots
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

encoder_2v2 = encoder_df.pivot(index='ROI', columns='Embedding', values='2v2_Accuracy')
encoder_2v2.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('ENCODER: 2v2 Accuracy', fontsize=14, fontweight='bold')
axes[0].set_ylabel('2v2 Accuracy')
axes[0].legend(title='Embedding', bbox_to_anchor=(1.05, 1))
axes[0].tick_params(axis='x', rotation=45)

encoder_pc = encoder_df.pivot(index='ROI', columns='Embedding', values='Pearson_Correlation')
encoder_pc.plot(kind='bar', ax=axes[1], width=0.8)
axes[1].set_title('ENCODER: Pearson Correlation', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Pearson Correlation')
axes[1].legend(title='Embedding', bbox_to_anchor=(1.05, 1))
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('encoder_results.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved encoder_results.png")
plt.close()

 Saving results and creating plots...
  ✓ Saved decoder_results.csv
  ✓ Saved encoder_results.csv
  ✓ Saved decoder_results.png
  ✓ Saved encoder_results.png


In [24]:
# =============================================================================
# SUMMARY
# =============================================================================

print(" DECODER - Embedding Rankings (by average score):")
decoder_avg = decoder_df.groupby('Embedding').agg({
    '2v2_Accuracy': 'mean',
    'Pearson_Correlation': 'mean'
})
decoder_avg['Average'] = (decoder_avg['2v2_Accuracy'] + decoder_avg['Pearson_Correlation']) / 2
print(decoder_avg.sort_values('Average', ascending=False).round(4))

print(" ENCODER - Embedding Rankings (by average score):")
encoder_avg = encoder_df.groupby('Embedding').agg({
    '2v2_Accuracy': 'mean',
    'Pearson_Correlation': 'mean'
})
encoder_avg['Average'] = (encoder_avg['2v2_Accuracy'] + encoder_avg['Pearson_Correlation']) / 2
print(encoder_avg.sort_values('Average', ascending=False).round(4))

 DECODER - Embedding Rankings (by average score):
                2v2_Accuracy  Pearson_Correlation  Average
Embedding                                                 
RoBERTa_CLS           0.9380               0.9975   0.9677
RoBERTa_Pooled        0.9204               0.9695   0.9449
BERT_CLS              0.8853               0.7875   0.8364
GloVe                 0.8923               0.7271   0.8097
BERT_Pooled           0.9413               0.6492   0.7952
 ENCODER - Embedding Rankings (by average score):
                2v2_Accuracy  Pearson_Correlation  Average
Embedding                                                 
RoBERTa_CLS           0.7929               0.5977   0.6953
RoBERTa_Pooled        0.7774               0.5412   0.6593
GloVe                 0.7393               0.5284   0.6338
BERT_Pooled           0.7109               0.4353   0.5731
BERT_CLS              0.6814               0.4148   0.5481
